In [1]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 2.9MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=2e33a8e2629f90cae9de859443e9a3b95925e16b7110cc2e47048f4ac7ab8c8e
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dafrel","key":"fe8efba2f9884818bb24c6c528cac916"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d ankur1809/breakhist-dataset

100% 7.92G/7.92G [03:04<00:00, 16.8MB/s]
100% 7.92G/7.92G [03:04<00:00, 46.2MB/s]


In [5]:
!unzip -uq "/content/breakhist-dataset.zip" -d "/content/proj"

In [6]:
#!ls "/content/proj"

In [1]:
import  os
import pandas as pd
import io
import tensorflow as tf
import gc
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tensorflow import keras
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD,RMSprop,Adam
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import shutil

In [9]:
os.listdir('/content/proj/BreakHist_Dataset')

['100X', '400X', '200X', '40X']

In [3]:
path='/content/proj/BreakHist_Dataset'


In [4]:
count=0
cnt=0
train=[]
val=[]
tl=[]
vl=[]
for i in os.listdir(path):
  for j in os.listdir(path+'/'+i):
    label=j
    for k in os.listdir(path+'/'+i+'/'+j):
      for l in os.listdir(path+'/'+i+'/'+j+'/'+k):
        
        if j =='Benign':
          img=cv2.imread(path+'/'+i+'/'+j+'/'+k+'/'+l)
          img=cv2.resize(img,(150,150))
          if count<1943:
            train.append(img)
            tl.append(0)
          else:
            val.append(img)
            vl.append(0)
          count+=1
        elif j=='Malignant':
          img=cv2.imread(path+'/'+i+'/'+j+'/'+k+'/'+l)
          img=cv2.resize(img,(150,150))
          if cnt<4343:
            train.append(img)
            tl.append(1)
          else:
            val.append(img)
            vl.append(1)
          cnt+=1
        


        

In [58]:
len(train)

6286

In [46]:
len()

3

In [5]:
t = np.array(train)
t = t.astype('float32')
t /= 255
 
labels = np.array(tl)
Y = np_utils.to_categorical(labels, 2)

In [6]:
tt = np.array(val)
tt = tt.astype('float32')
tt /= 255

labels = np.array(vl)
Yt = np_utils.to_categorical(labels, 2)

For complete training data


In [7]:
x,y = shuffle(t,Y, random_state=1)
x = x.reshape(-1,150, 150, 3) 



In [8]:
v,yl = shuffle(tt,Yt, random_state=1)
v = v.reshape(-1,150, 150, 3) 

In [9]:
model=Sequential()
model.add(Conv2D(256,(3,3),input_shape=(150,150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (2, 2)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(256, (1, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dense(300))
model.add(Activation('relu'))
model.add(Dense(500))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])

In [10]:
hist = model.fit(x, y, batch_size=30,epochs=30, validation_data=(v,yl))

Epoch 1/30
210/210 [==============================] - 25s 118ms/step - loss: 0.5484 - accuracy: 0.7374 - val_loss: 0.6309 - val_accuracy: 0.6069
Epoch 2/30
210/210 [==============================] - 24s 117ms/step - loss: 0.5571 - accuracy: 0.7315 - val_loss: 0.5650 - val_accuracy: 0.6959
Epoch 3/30
210/210 [==============================] - 25s 119ms/step - loss: 0.4994 - accuracy: 0.7821 - val_loss: 0.5812 - val_accuracy: 0.7017
Epoch 4/30
210/210 [==============================] - 25s 118ms/step - loss: 0.4751 - accuracy: 0.8037 - val_loss: 0.5212 - val_accuracy: 0.7805
Epoch 5/30
210/210 [==============================] - 25s 117ms/step - loss: 0.4790 - accuracy: 0.7991 - val_loss: 0.5027 - val_accuracy: 0.7971
Epoch 6/30
210/210 [==============================] - 25s 118ms/step - loss: 0.4470 - accuracy: 0.8236 - val_loss: 0.4742 - val_accuracy: 0.7996
Epoch 7/30
210/210 [==============================] - 25s 117ms/step - loss: 0.4557 - accuracy: 0.8131 - val_loss: 0.4919 - val_ac

In [11]:
pre=model.predict_classes(v)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [12]:
y1 = np.argmax(yl, axis=-1)

In [13]:
from sklearn.metrics import classification_report, confusion_matrix

In [14]:
print( confusion_matrix(y_true=y1,y_pred= pre))

[[ 267  219]
 [  57 1029]]


In [15]:
report = classification_report(y1, pre)
print(report)

              precision    recall  f1-score   support

           0       0.82      0.55      0.66       486
           1       0.82      0.95      0.88      1086

    accuracy                           0.82      1572
   macro avg       0.82      0.75      0.77      1572
weighted avg       0.82      0.82      0.81      1572

